## Installation & Loading Packages

In [ ]:
#@title install packages
!pip install transformers
!pip install transformers[sentencepiece]
!pip install -U sentence-transformers
!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu116
!pip install gradio -q #interface
#!pip install datasets
#!pip install transformers datasets accelerate nvidia-ml-py3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 31.5 MB/s 
     |████████████████████████████████| 7.6 MB 55.6 MB/s 
     |████████████████████████████████| 163 kB 67.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 11.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 3.4 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=be2bd5230d3b3bd5d120f77ed7e85fd1665e64652f7d37f090b9c5c7a83f0031
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/pu

In [ ]:
#@title Import Packages-Summarizer

import warnings
warnings.filterwarnings("ignore")
from transformers import PegasusForConditionalGeneration, PegasusTokenizer,AutoTokenizer, AutoModel
from transformers import pipeline
import torch
import requests
from PIL import Image
import glob 
import cv2
from os import listdir
import sys
#from pynvml import *

In [ ]:
#@title Image Caption Model Installment
! pip install git+https://github.com/openai/CLIP.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-hni0wjpn
  Running command git clone -q https://github.com/openai/CLIP.git /tmp/pip-req-build-hni0wjpn
     |████████████████████████████████| 53 kB 1.6 MB/s 
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369409 sha256=f1c3f56890ba6b8d116f09ca0ec765308d69cfd5bd035e1dadfde82e17366454
  Stored in directory: /tmp/pip-ephem-wheel-cache-dkrs39db/wheels/fd/b9/c3/5b4470e35ed76e174bff77c92f91da82098d5e35fd5bc8cdac
Successfully built clip


In [ ]:
#@title Define Functions for Generator
import os
from huggingface_hub import hf_hub_download
conceptual_weight = hf_hub_download(repo_id="akhaliq/CLIP-prefix-captioning-conceptual-weights", filename="conceptual_weights.pt")
coco_weight = hf_hub_download(repo_id="akhaliq/CLIP-prefix-captioning-COCO-weights", filename="coco_weights.pt")
import clip
import os
from torch import nn
import numpy as np
import torch
import torch.nn.functional as nnf
import sys
from typing import Tuple, List, Union, Optional
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm, trange
import skimage.io as io
import PIL.Image
import gradio as gr

N = type(None)
V = np.array
ARRAY = np.ndarray
ARRAYS = Union[Tuple[ARRAY, ...], List[ARRAY]]
VS = Union[Tuple[V, ...], List[V]]
VN = Union[V, N]
VNS = Union[VS, N]
T = torch.Tensor
TS = Union[Tuple[T, ...], List[T]]
TN = Optional[T]
TNS = Union[Tuple[TN, ...], List[TN]]
TSN = Optional[TS]
TA = Union[T, ARRAY]


D = torch.device
CPU = torch.device('cpu')


def get_device(device_id: int) -> D:
    if not torch.cuda.is_available():
        return CPU
    device_id = min(torch.cuda.device_count() - 1, device_id)
    return torch.device(f'cuda:{device_id}')


CUDA = get_device

class MLP(nn.Module):

    def forward(self, x: T) -> T:
        return self.model(x)

    def __init__(self, sizes: Tuple[int, ...], bias=True, act=nn.Tanh):
        super(MLP, self).__init__()
        layers = []
        for i in range(len(sizes) -1):
            layers.append(nn.Linear(sizes[i], sizes[i + 1], bias=bias))
            if i < len(sizes) - 2:
                layers.append(act())
        self.model = nn.Sequential(*layers)


class ClipCaptionModel(nn.Module):

    #@functools.lru_cache #FIXME
    def get_dummy_token(self, batch_size: int, device: D) -> T:
        return torch.zeros(batch_size, self.prefix_length, dtype=torch.int64, device=device)

    def forward(self, tokens: T, prefix: T, mask: Optional[T] = None, labels: Optional[T] = None):
        embedding_text = self.gpt.transformer.wte(tokens)
        prefix_projections = self.clip_project(prefix).view(-1, self.prefix_length, self.gpt_embedding_size)
        #print(embedding_text.size()) #torch.Size([5, 67, 768])
        #print(prefix_projections.size()) #torch.Size([5, 1, 768])
        embedding_cat = torch.cat((prefix_projections, embedding_text), dim=1)
        if labels is not None:
            dummy_token = self.get_dummy_token(tokens.shape[0], tokens.device)
            labels = torch.cat((dummy_token, tokens), dim=1)
        out = self.gpt(inputs_embeds=embedding_cat, labels=labels, attention_mask=mask)
        return out

    def __init__(self, prefix_length: int, prefix_size: int = 512):
        super(ClipCaptionModel, self).__init__()
        self.prefix_length = prefix_length
        self.gpt = GPT2LMHeadModel.from_pretrained('gpt2')
        self.gpt_embedding_size = self.gpt.transformer.wte.weight.shape[1]
        if prefix_length > 10:  # not enough memory
            self.clip_project = nn.Linear(prefix_size, self.gpt_embedding_size * prefix_length)
        else:
            self.clip_project = MLP((prefix_size, (self.gpt_embedding_size * prefix_length) // 2, self.gpt_embedding_size * prefix_length))


class ClipCaptionPrefix(ClipCaptionModel):

    def parameters(self, recurse: bool = True):
        return self.clip_project.parameters()

    def train(self, mode: bool = True):
        super(ClipCaptionPrefix, self).train(mode)
        self.gpt.eval()
        return self
        

def generate_beam(model, tokenizer, beam_size: int = 5, prompt=None, embed=None,
                  entry_length=67, temperature=1., stop_token: str = '.'):

    model.eval()
    stop_token_index = tokenizer.encode(stop_token)[0]
    tokens = None
    scores = None
    device = next(model.parameters()).device
    seq_lengths = torch.ones(beam_size, device=device)
    is_stopped = torch.zeros(beam_size, device=device, dtype=torch.bool)
    with torch.no_grad():
        if embed is not None:
            generated = embed
        else:
            if tokens is None:
                tokens = torch.tensor(tokenizer.encode(prompt))
                tokens = tokens.unsqueeze(0).to(device)
                generated = model.gpt.transformer.wte(tokens)
        for i in range(entry_length):
            outputs = model.gpt(inputs_embeds=generated)
            logits = outputs.logits
            logits = logits[:, -1, :] / (temperature if temperature > 0 else 1.0)
            logits = logits.softmax(-1).log()
            if scores is None:
                scores, next_tokens = logits.topk(beam_size, -1)
                generated = generated.expand(beam_size, *generated.shape[1:])
                next_tokens, scores = next_tokens.permute(1, 0), scores.squeeze(0)
                if tokens is None:
                    tokens = next_tokens
                else:
                    tokens = tokens.expand(beam_size, *tokens.shape[1:])
                    tokens = torch.cat((tokens, next_tokens), dim=1)
            else:
                logits[is_stopped] = -float(np.inf)
                logits[is_stopped, 0] = 0
                scores_sum = scores[:, None] + logits
                seq_lengths[~is_stopped] += 1
                scores_sum_average = scores_sum / seq_lengths[:, None]
                scores_sum_average, next_tokens = scores_sum_average.view(-1).topk(beam_size, -1)
                next_tokens_source = next_tokens // scores_sum.shape[1]
                seq_lengths = seq_lengths[next_tokens_source]
                next_tokens = next_tokens % scores_sum.shape[1]
                next_tokens = next_tokens.unsqueeze(1)
                tokens = tokens[next_tokens_source]
                tokens = torch.cat((tokens, next_tokens), dim=1)
                generated = generated[next_tokens_source]
                scores = scores_sum_average * seq_lengths
                is_stopped = is_stopped[next_tokens_source]
            next_token_embed = model.gpt.transformer.wte(next_tokens.squeeze()).view(generated.shape[0], 1, -1)
            generated = torch.cat((generated, next_token_embed), dim=1)
            is_stopped = is_stopped + next_tokens.eq(stop_token_index).squeeze()
            if is_stopped.all():
                break
    scores = scores / seq_lengths
    output_list = tokens.cpu().numpy()
    output_texts = [tokenizer.decode(output[:int(length)]) for output, length in zip(output_list, seq_lengths)]
    order = scores.argsort(descending=True)
    output_texts = [output_texts[i] for i in order]
    return output_texts


def generate2(
        model,
        tokenizer,
        tokens=None,
        prompt=None,
        embed=None,
        entry_count=1,
        entry_length=67,  # maximum number of words
        top_p=0.8,
        temperature=1.,
        stop_token: str = '.',
):
    model.eval()
    generated_num = 0
    generated_list = []
    stop_token_index = tokenizer.encode(stop_token)[0]
    filter_value = -float("Inf")
    device = next(model.parameters()).device

    with torch.no_grad():

        for entry_idx in trange(entry_count):
            if embed is not None:
                generated = embed
            else:
                if tokens is None:
                    tokens = torch.tensor(tokenizer.encode(prompt))
                    tokens = tokens.unsqueeze(0).to(device)

                generated = model.gpt.transformer.wte(tokens)

            for i in range(entry_length):

                outputs = model.gpt(inputs_embeds=generated)
                logits = outputs.logits
                logits = logits[:, -1, :] / (temperature if temperature > 0 else 1.0)
                sorted_logits, sorted_indices = torch.sort(logits, descending=True)
                cumulative_probs = torch.cumsum(nnf.softmax(sorted_logits, dim=-1), dim=-1)
                sorted_indices_to_remove = cumulative_probs > top_p
                sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[
                                                    ..., :-1
                                                    ].clone()
                sorted_indices_to_remove[..., 0] = 0

                indices_to_remove = sorted_indices[sorted_indices_to_remove]
                logits[:, indices_to_remove] = filter_value
                next_token = torch.argmax(logits, -1).unsqueeze(0)
                next_token_embed = model.gpt.transformer.wte(next_token)
                if tokens is None:
                    tokens = next_token
                else:
                    tokens = torch.cat((tokens, next_token), dim=1)
                generated = torch.cat((generated, next_token_embed), dim=1)
                if stop_token_index == next_token.item():
                    break

            output_list = list(tokens.squeeze().cpu().numpy())
            output_text = tokenizer.decode(output_list)
            generated_list.append(output_text)

    return generated_list[0]


Downloading:   0%|          | 0.00/636M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/636M [00:00<?, ?B/s]

In [ ]:
#@title Image Caption Generator

is_gpu = False 
device = CUDA(0) if is_gpu else "cpu"
clip_model, preprocess = clip.load("ViT-B/32", device=device, jit=False)
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

def prediction(img,model_name):   
  prefix_length = 10
    
  model = ClipCaptionModel(prefix_length)
    
  if model_name == "COCO":
      model_path = coco_weight
  else:
      model_path = conceptual_weight
  model.load_state_dict(torch.load(model_path, map_location=CPU)) 
  model = model.eval() 
  device = CUDA(0) if is_gpu else "cpu"
  model = model.to(device)

  use_beam_search = False 
  #image = io.imread(img)
  image=cv2.imread(img)
  pil_image = PIL.Image.fromarray(image)  
  image = preprocess(pil_image).unsqueeze(0).to(device)
  with torch.no_grad():
      prefix = clip_model.encode_image(image).to(device, dtype=torch.float32)
      prefix_embed = model.clip_project(prefix).reshape(1, prefix_length, -1)
  if use_beam_search:
      generated_text_prefix = generate_beam(model, tokenizer, embed=prefix_embed)[0]
  else:
      generated_text_prefix = generate2(model, tokenizer, embed=prefix_embed)
  return generated_text_prefix

100%|████████████████████████████████████████| 338M/338M [00:02<00:00, 145MiB/s]


Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

@article{mokady2021clipcap, title={ClipCap: CLIP Prefix for Image Captioning}, author={Mokady, Ron and Hertz, Amir and Bermano, Amit H}, journal={arXiv preprint arXiv:2111.09734}, year={2021} }
https://huggingface.co/spaces/akhaliq/CLIP_prefix_captioning/blob/main/app.py

## Paraphrase&Similarity


Models

In [ ]:
from sentence_transformers import SentenceTransformer, util
model_sen = SentenceTransformer('stsb-roberta-large')
#embeddings = model.encode(txt3, convert_to_tensor=True)

Downloading:   0%|          | 0.00/748 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/191 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.92k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/674 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_pa = AutoModelForSeq2SeqLM.from_pretrained("ramsrigouthamg/t5-large-paraphraser-diverse-high-quality")
tokenizer_pa = AutoTokenizer.from_pretrained("ramsrigouthamg/t5-large-paraphraser-diverse-high-quality")

import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#print ("device ",device)
model_pa = model_pa.to(device)

Downloading:   0%|          | 0.00/1.32k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.95G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.86k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

 Function

In [ ]:
def sen_similarity(text):
  embeddings = model_sen.encode(text, convert_to_tensor=True)
  txt=[]
  for i in range(0,len(text)):
    for j in range(0,len(text)):
      sim=util.pytorch_cos_sim(embeddings[i], embeddings[j])
      if sim<0.6:
        if not txt:
          txt.append(text[i])
          txt.append(text[j])
        else:
          if text[i] not in txt:
            txt.append(text[i])
            if text[j] not in txt:
              txt.append(text[j])
  return txt

In [ ]:
import random
from random import sample
def text_modifier(text):
 # text_1=random.sample(text,19)
  text_1="".join(text_1)
  return text_1

In [ ]:
def packtext(text):
  N=5
  textlist=list(zip(*([iter(text)] * N)))
  return textlist

In [ ]:
def convertTuple(tup):
    st = ''.join(tup)
    return st

In [ ]:
def paraphrase(text):
  para=[]  
  textlist=packtext(text)
  for i in range(0,len(textlist)):   
    str=convertTuple(textlist[i])
    text_1="paraphrase: "+str + " </s>"
    encoding = tokenizer_pa.encode_plus(text_1,max_length =128, padding=True, return_tensors="pt")
    input_ids,attention_mask  = encoding["input_ids"].to(device), encoding["attention_mask"].to(device)

    model_pa.eval()
    beam_outputs = model_pa.generate(
        input_ids=input_ids,attention_mask=attention_mask,
        max_length=128,
        early_stopping=True,
        num_beams=10,
        num_return_sequences=1 )
    for beam_output in beam_outputs:
      sent = tokenizer_pa.decode(beam_output, skip_special_tokens=True,clean_up_tokenization_spaces=True)
      para.append(sent)
  
    list1=[i.split(':')[1] for i in para]
    Paratext= "".join(list1)

  return Paratext

##Image Caption Summarizer

In [ ]:
def get_image_caption_summary (path,model,loc):
  
  fileList=os.listdir(path)
  images =[cv2.imread(file) for file in fileList]
  n=len(fileList)
  text=[]
  for i in range(0,n):
    # image=transform(img)
    if images[i] is not None:
      pred=prediction(fileList[i],model)
      text.append(pred)
    else:
      error=str(os.path.basename(fileList[i]))
      print(error+' '+':'+'is None')
      continue
  text_sim=sen_similarity(text)
  paraptext=paraphrase(text_sim)
  summarizer = pipeline("summarization", model=loc,batch_size=10)
  summary=summarizer(paraptext, max_length=130, min_length=10, do_sample=True)
  return summary

## Test

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd '/content/drive/MyDrive/dataset test/Mon&Dau_000'

/content/drive/MyDrive/dataset test/Mon&Dau_000


In [ ]:
dir="/content/drive/MyDrive/dataset test/Mon&Dau_000"

In [ ]:
#Summarizer Model:"facebook/bart-large-xsum" / "knkarthick/MEETING_SUMMARY" / "google/pegasus-xsum"
loc="facebook/bart-large-xsum"
#Image Cap Model: "COCO" or "CO" -anything will trigger Conceptual 

In [ ]:
txt3=get_image_caption_summary(dir,"CO",loc)
txt3

100%|██████████| 1/1 [00:01<00:00,  1.47s/it]


新建 Microsoft Excel 工作表.xlsx :is None


[{'summary_text': "A selection of photographs from the BBC's documentary series The Art of the Blue."}]